# User Behavior Overview

## Imports

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from vis_plotly import *

## Data reading

In [4]:
df = pd.read_csv("../data/clean_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146891 entries, 0 to 146890
Data columns (total 34 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   bearer_id                146891 non-null  float64
 1   msisdn_number            146891 non-null  float64
 2   imei                     146891 non-null  float64
 3   imsi                     146891 non-null  float64
 4   last_location_name       146891 non-null  object 
 5   handset_manufacturer     146891 non-null  object 
 6   handset_type             146891 non-null  object 
 7   start                    146891 non-null  object 
 8   end                      146891 non-null  object 
 9   dur_(ms)                 146891 non-null  float64
 10  social_media_dl_(bytes)  146891 non-null  float64
 11  social_media_ul_(bytes)  146891 non-null  float64
 12  google_dl_(bytes)        146891 non-null  float64
 13  google_ul_(bytes)        146891 non-null  float64
 14  emai

##  Top 10 handsets used by the customers

In [5]:
class DfSelector():

    def __init__(self):
        pass

    def filter_by_count(self, df, column):
      new_df = df[column].value_counts(ascending=False).reset_index().copy()
      new_df = new_df.rename(
          columns={'index': column, column: "count"})
      return new_df

    def filter_by_sum(self, df, column):
      new_df = df[column].sum(ascending=False).reset_index().copy()

      new_df = new_df.rename(
          columns={'index': column, column: "sum"})
      return new_df

    
    def find_agg(self, df: pd.DataFrame, agg_column: str, agg_metric: str, col_name: str, top: int, order=False) -> pd.DataFrame:
        new_df = df.groupby(agg_column)[agg_column].agg(agg_metric).reset_index(name=col_name).\
        sort_values(by=col_name, ascending=order)[:top]
        return new_df
    
selector = DfSelector()


In [6]:
count_df = selector.filter_by_count(df, 'handset_type')
count_df.head(10)

,handset_type,count
0,Huawei B528S-23A,28193
1,Apple iPhone 6S (A1688),9369
2,Apple iPhone 6 (A1586),8967
3,Apple iPhone 7 (A1778),6240
4,Apple iPhone Se (A1723),5151
5,Apple iPhone 8 (A1905),4961
6,Apple iPhone Xr (A2105),4542
7,Samsung Galaxy S8 (Sm-G950F),4404
8,Apple iPhone X (A1901),3788
9,Samsung Galaxy A5 Sm-A520F,3673


In [7]:
fig = px.histogram(count_df.nlargest(10, 'count'), x="handset_type", y='count')
fig.show()

## Top 3 handset manufacturers

In [8]:
top_manufacturers = selector.filter_by_count(df, 'handset_manufacturer')
top_manufacturers.head(3)

,handset_manufacturer,count
0,Apple,67892
1,Samsung,39902
2,Huawei,33755


##  Top 5 handsets per top 3 handset manufacturer

In [9]:
top3_manufacturers = top_manufacturers.head(3)['handset_manufacturer'].values
top_manufacturers = df[df["handset_manufacturer"].isin(top3_manufacturers)]
top_manufacturers['handset_type'].groupby(
    df['handset_manufacturer']).apply(lambda x: x.value_counts().head(5))

handset_manufacturer                                
Apple                 Apple iPhone 6S (A1688)            9369
                      Apple iPhone 6 (A1586)             8967
                      Huawei B528S-23A                   8820
                      Apple iPhone 7 (A1778)             6240
                      Apple iPhone Se (A1723)            5151
Huawei                Huawei B528S-23A                  19373
                      Huawei E5180                       2062
                      Huawei P20 Lite Huawei Nova 3E     2000
                      Huawei P20                         1471
                      Huawei Y6 2018                      984
Samsung               Samsung Galaxy S8 (Sm-G950F)       4404
                      Samsung Galaxy A5 Sm-A520F         3673
                      Samsung Galaxy J5 (Sm-J530)        3664
                      Samsung Galaxy J3 (Sm-J330)        3431
                      Samsung Galaxy S7 (Sm-G930X)       3106
Name: handset_typ

# Task 1.1

## Number of xDR sessions

In [10]:
sessions = selector.find_agg(df, 'msisdn_number', 'count', 'bearer_id', 10, False)
sessions

,msisdn_number,bearer_id
13381,33626320676.00,18
6353,33614892860.00,17
13037,33625779332.00,17
36669,33659725664.00,16
75564,33675877202.00,15
91949,33760536639.00,15
64472,33667163239.00,13
653,33603127838.00,12
91608,33760413819.00,12
13845,33627080969.00,12


## Session duration

In [38]:
duration = df.groupby('msisdn_number').agg({'dur_(ms)': 'sum'})
duration = duration.rename(columns={'dur_(ms)': 'total_duration(ms)'})
duration.sort_values(by=['total_duration(ms)'], ascending=False).head(10)

,total_duration(ms)
msisdn_number,
33625779332.00,4138200.00
33626320676.00,4007173.50
33614892860.00,3900151.50
33659725664.00,3492445.50
33760536639.00,3396748.25
33675877202.00,3332098.50
33667163239.00,3114064.00
33603127838.00,2855400.25
33627080969.00,2782307.50


In [39]:
duration.describe()

,total_duration(ms)
count,105719.00
mean,139206.08
std,129415.13
min,7142.00
25%,72729.00
50%,103111.00
75%,172799.00
max,4138200.00


## The total download (DL) and upload (UL) data

In [13]:
usage = df.groupby('msisdn_number')[['total_dl_(bytes)', 'total_ul_(bytes)']].sum()
usage['total_data'] = usage['total_dl_(bytes)'] + usage['total_ul_(bytes)']
usage.nlargest(20, 'total_data')

,total_dl_(bytes),total_ul_(bytes),total_data
msisdn_number,,,
33614892860.00,8156743493.00,689483001.00,8846226494.00
33760536639.00,7811295382.00,703478581.00,8514773963.00
33625779332.00,7770043342.00,729577380.00,8499620722.00
33626320676.00,7301516540.00,669650721.00,7971167261.00
33675877202.00,7309541816.00,581568792.00,7891110608.00
33659725664.00,7081602462.00,624260321.00,7705862783.00
33666464084.00,6903439962.00,405060976.00,7308500938.00
33760413819.00,6610851624.00,521518890.00,7132370514.00
33664712899.00,6400773755.00,471244453.00,6872018208.00


In [14]:
usage.describe()

,total_dl_(bytes),total_ul_(bytes),total_data
count,105719.00,105719.00,105719.00
mean,631730049.57,57136009.85,688866059.42
std,460920903.80,35271796.89,486996435.35
min,8827082.00,2866892.00,33249009.00
25%,314240570.50,36378493.50,357950085.00
50%,569379683.00,46762975.00,616933544.00
75%,806298148.00,65504102.50,856473809.50
max,8156743493.00,729577380.00,8846226494.00


## The total data volume (in Bytes) during this session for each application

In [15]:
df["social_media"] = df["social_media_dl_(bytes)"] + df['social_media_ul_(bytes)']
df["google"] = df["google_dl_(bytes)"] + df["google_ul_(bytes)"]
df['email'] = df["email_dl_(bytes)"] + df["email_ul_(bytes)"]
df['youtube'] = df["youtube_dl_(bytes)"] + df["youtube_ul_(bytes)"]
df['netflix'] = df["netflix_dl_(bytes)"] + df["netflix_ul_(bytes)"]
df["gaming"] = df["gaming_dl_(bytes)"] + df["gaming_ul_(bytes)"]
df['other'] = df["other_dl_(bytes)"]+df["other_ul_(bytes)"]
df['total_data'] = df['total_dl_(bytes)'] + df['total_ul_(bytes)']

In [16]:
apps_df = df.groupby('msisdn_number')[['social_media', 'google', 'email', 'youtube', 'gaming', 'other', 'total_data']].sum().nlargest(20, 'total_data')
apps_df

,social_media,google,email,youtube,gaming,other,total_data
msisdn_number,,,,,,,
33614892860.00,28294544.00,127973787.00,40788634.00,394370218.00,7749432234.00,7639263572.00,8846226494.00
33760536639.00,39783189.00,123223099.00,33693767.00,396289198.00,7461045228.00,4716134493.00,8514773963.00
33625779332.00,27135500.00,142307915.00,40633966.00,452958769.00,7326673487.00,6354583086.00,8499620722.00
33626320676.00,43374779.00,152191852.00,42418782.00,374483047.00,6887572116.00,8167877776.00,7971167261.00
33675877202.00,19222921.00,109860502.00,31514421.00,317410572.00,6970567597.00,6798515150.00,7891110608.00
33659725664.00,35412358.00,116516345.00,35999792.00,257991088.00,6725559211.00,6317415487.00,7705862783.00
33666464084.00,18629986.00,89320737.00,25557139.00,227336012.00,6646303338.00,4349141478.00,7308500938.00
33760413819.00,20777205.00,82738720.00,25576965.00,303169107.00,6268619592.00,4101645436.00,7132370514.00
33664712899.00,19390599.00,90389372.00,21426007.00,276834013.00,6103856008.00,3976960308.00,6872018208.00


In [17]:
df.to_csv('../data/clean_data.csv', index=False)